In [ ]:
#default_exp web.demo

# Web Demo

> Simple demo UI.

In [ ]:
#export
from vulcan_medication_bundle.per_patient import *

from flask import Blueprint
from flask import flash
from flask import g
from flask import redirect
from flask import render_template
from flask import request
from flask import url_for
from flask import send_from_directory
from werkzeug.exceptions import abort
from werkzeug.utils import secure_filename
from io import StringIO
from pathlib import Path
import tempfile,time,json,requests
import pandas as pd

bp = Blueprint("demo", __name__)

In [ ]:
#export
def _get_single_patient_medication_bundle(api_base, subject):
    bundle = create_single_patient_medication_bundle(api_base, subject)
    bundle = handle_entry_search(bundle)
#     bundle = filter_bundle(bundle, medication_status_filter)
#     bundle = filter_bundle(bundle, do_not_perform_filter)
    return bundle

In [ ]:
#export
@bp.route("/", methods=(["GET"]))
def index():
    """Show the index page."""
    return render_template("index.html", medication_bundle=None)

In [ ]:
#export
@bp.route("/get_single_patient_medication_bundle", methods=(["GET", "POST"]))
def get_single_patient_medication_bundle():
    # TODO: we have v similar endpoint in app )o:
    medication_bundle = ''
    if request.method == "POST":
        api_base = request.form["api_base"]
        subject = request.form["subject"]
        headers = request.form["headers"]
        print(api_base, subject) # DO NOT log headers - might contain user secrets
        error = None

        if not api_base:
            error = "api_base is required."
        if not subject:
            error = "subject is required"

        import vulcan_medication_bundle
        if headers:
            print('Using custom headers')
            vulcan_medication_bundle.core.request_headers = json.loads(headers)
        else:
            print('no custom headers')
            vulcan_medication_bundle.core.request_headers = {}
            
        if error is not None:
            flash(error)
        else:
            medication_bundle = _get_single_patient_medication_bundle(api_base, subject)
            medication_bundle = json.dumps(medication_bundle, indent=2)
    
    return render_template("index.html", medication_bundle=medication_bundle)

In [ ]:
#export
@bp.route("/convert_to_cdisc", methods=(["POST"]))
def convert_to_cdisc():
    medication_bundle = request.form["medication_bundle"]
    cdisc_response = requests.post(
            'https://mylinks-prod-sdtmtool.azurewebsites.net/TransformBundle/Process', 
            json=json.loads(medication_bundle))
    print('Got CDISC response', cdisc_response, cdisc_response.json())
    cdisc_json = cdisc_response.json()
    dfs = [pd.read_csv(StringIO(cdisc_json[k])) for k in cdisc_json]
    dfs = [df.dropna(axis=1) for df in dfs]
    cdisc_tables = [df.to_html(classes='data', header="true") for df in dfs]
    
    return render_template("index.html", medication_bundle=medication_bundle, cdisc_tables=cdisc_tables)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 10_per_patient.ipynb.
Converted 20a_status_filter.ipynb.
Converted 30_cli.ipynb.
Converted 50_web_app.ipynb.
Converted 50a_web_demo.ipynb.
Converted index.ipynb.
